# Fase 1 – Creación de Archivos
Crear carpetas data, database, reports, ci, git y scripts y estructura base del proyecto usando os y pathlib en tiempo de ejecución


In [1]:
from pathlib import Path

# Crear estructura base
base_path = Path("SaludDigitalB_E1")

folders = [
    "data", "database", "reports", "ci", "git", "scripts"
]

for folder in folders:
    (base_path / folder).mkdir(parents=True, exist_ok=True)

print("Estructura creada correctamente ✅")


Estructura creada correctamente ✅


# Fase 2 – Definición de Esquema
Generar un archivo base CSV (base.csv) con la estructura indicada.


In [2]:
import pandas as pd

columnas = [
    "id_paciente", "nombre", "edad", "sexo",
    "distrito", "servicio", "fecha_atencion", "monto"
]

df_base = pd.DataFrame(columns=columnas)
df_base.to_csv("SaludDigitalB_E1/data/base.csv", index=False)

print("Archivo base.csv creado correctamente ✅")


Archivo base.csv creado correctamente ✅


# Fase 3 – Generación de Datos Aleatorios
Generar un archivo CSV (pacientes.csv) con 3000 registros simulados, que incluya campos nulos, NA, entre otros


In [7]:
# ...existing code...
import pandas as pd
import numpy as np
import random
from faker import Faker

fake = Faker('es_ES')
servicios = ["Odontología", "Pediatría", "Medicina General", "Cardiología", "Dermatología"]
distritos = ["Miraflores", "Surco", "San Isidro", "La Molina", "Barranco"]

data = []
for i in range(3000):
    data.append([
        f"P{i+1:04d}",
        fake.name(),
        np.random.randint(1, 90),
        random.choice(["M", "F"]),
        random.choice(distritos),
        random.choice(servicios),
        fake.date_between(start_date="-2y", end_date="today"),
        round(np.random.uniform(50, 500), 2)
    ])

df = pd.DataFrame(data, columns=[
    "id_paciente", "nombre", "edad", "sexo",
    "distrito", "servicio", "fecha_atencion", "monto"
])

# Introducir valores nulos aleatorios
for col in ["sexo", "distrito"]:
    df.loc[df.sample(frac=0.02, random_state=1).index, col] = np.nan

df.to_csv("SaludDigitalB_E1/data/pacientes.csv", index=False)
print("Archivo pacientes.csv generado correctamente ✅")


Archivo pacientes.csv generado correctamente ✅
